## Deposits Rates Buckets Test

### 1. Import the libraries needed

In [1]:
import pandas as pd
from pandas.tseries.offsets import MonthEnd
import numpy as np
from bs4 import BeautifulSoup
import requests
pd.__version__

'0.18.1'

### 2. Define the input files and urls

In [2]:
!pwd

/home/martin/PycharmProjects/mas


In [3]:
depositAmountsFile = 'files/Deposit_amounts.xlsx'
depositsRatesBuckets = 'files/Deposits_rates_and_buckets.xlsx'
url = '''http://www.cbr.ru/eng/statistics/print.aspx?file=credit_statistics/ex_rate_ind_16_e.htm&pid=svs&sid=analit'''

### 3. Read file Deposit_amounts.xlsx and mangle and align the data for further calculations, skip first 7 rows, don't put headers

In [4]:
dfDepositAmounts = pd.read_excel(depositAmountsFile, sheetname=0, skiprows=7, header=None)

#### Explore the data

In [5]:
dfDepositAmounts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 11 columns):
0     55 non-null datetime64[ns]
1     55 non-null int64
2     55 non-null int64
3     55 non-null int64
4     55 non-null int64
5     55 non-null int64
6     55 non-null int64
7     55 non-null int64
8     55 non-null int64
9     55 non-null int64
10    55 non-null int64
dtypes: datetime64[ns](1), int64(10)
memory usage: 4.8 KB


In [6]:
dfDepositAmounts.head()

,0,1,2,3,4,5,6,7,8,9,10
0,2012-02-01,19729592,6135566,419487,28841,4164728,758418,4595876,1723058,9528875,2101759
1,2012-03-01,19680942,6029030,435477,34525,4039083,760463,4501438,1619101,9700707,2088085
2,2012-04-01,19890777,6092441,479256,32655,4024004,734904,4577416,1648916,9807289,2164900
3,2012-05-01,20007143,6321617,470609,29689,3757078,686025,4684425,1779641,10075413,2168096
4,2012-06-01,20404098,6849573,463338,28928,3874913,761163,4776341,1911771,10161131,2342464


#### Take only the first and the last 4 columns from the dataset

In [7]:
dfDepositAmounts = dfDepositAmounts[[0, 7, 8, 9, 10]]

In [8]:
dfDepositAmounts.columns

Int64Index([0, 7, 8, 9, 10], dtype='int64')

#### Rename the columns in the dataset

In [9]:
dfDepositAmounts.columns = ['dt', 'entities_rub', 'entities_usd', 'individual_rub', 'individual_usd']

In [10]:
dfDepositAmounts.head()

,dt,entities_rub,entities_usd,individual_rub,individual_usd
0,2012-02-01,4595876,1723058,9528875,2101759
1,2012-03-01,4501438,1619101,9700707,2088085
2,2012-04-01,4577416,1648916,9807289,2164900
3,2012-05-01,4684425,1779641,10075413,2168096
4,2012-06-01,4776341,1911771,10161131,2342464


#### Filter out rows which have date older then 1 Jan 2016

In [11]:
dfDepositAmounts[dfDepositAmounts.dt >= '2016-01-01']

,dt,entities_rub,entities_usd,individual_rub,individual_usd
47,2016-01-01,6857658,6293696,16347070,6912395
48,2016-02-01,6724597,6099610,15879147,6977848
49,2016-03-01,6704958,5782209,16188382,6899635
50,2016-04-01,6482520,5157832,16338734,6287100
51,2016-05-01,7233984,4993260,16719956,6061482
52,2016-06-01,7294002,5037593,16846688,6177303
53,2016-07-01,7247267,4813989,17109434,6057397
54,2016-08-01,7639472,4695342,17209103,6355935


In [12]:
dfDepositAmounts = dfDepositAmounts[dfDepositAmounts.dt >= '2016-01-01']

#### Create a column with full month name from the dt column

In [13]:
dfDepositAmounts['month'] = dfDepositAmounts.dt.apply('{:%B}'.format)

In [14]:
dfDepositAmounts.sample(2)

,dt,entities_rub,entities_usd,individual_rub,individual_usd,month
53,2016-07-01,7247267,4813989,17109434,6057397,July
49,2016-03-01,6704958,5782209,16188382,6899635,March


#### Set column month to be an index for the dataset <br>(To be used to join with datasets from the second file and from the url)

In [15]:
dfDepositAmounts = dfDepositAmounts.set_index('month')

In [16]:
# del(dfDepositAmounts['dt'])

In [17]:
dfDepositAmounts.sample(2)

,dt,entities_rub,entities_usd,individual_rub,individual_usd
month,,,,,
July,2016-07-01,7247267,4813989,17109434,6057397
January,2016-01-01,6857658,6293696,16347070,6912395


### 4. Read file Deposits_rates_and_buckets.xlsx and mangle and align the data for further calculations
#### Since the file contains multiple spreadsheets read the first 2 sheets and populate them into a dictionary of dataframes. Skip the first 6 rows and don't use headers

In [18]:
dfdepositsRatesBuckets = pd.read_excel(depositsRatesBuckets, sheetname=[0, 1], skiprows=6, header=None)

#### Create a MultIindex columns because the dataframe in each sheet has repeating column names

In [19]:
idx = pd.MultiIndex.from_tuples([('individual', c) for c in dfdepositsRatesBuckets[0].columns[:12]] + 
                                 [('entity', c) for c in dfdepositsRatesBuckets[0].columns[12:]])

#### Define the column names for level 1 of the MultiIndex

In [20]:
cols = ['month', 
'demand deposits', 
'up to 30 days (including demand deposits)',
'up to 30 days (except demand deposits)',
'31 to 90 days',
'91 to 180 days',
'181 days to 1 year',
'up to 1 year (including demand deposits)',
'up to 1 year (except demand deposits',
'1 to 3 years',
'over 3 years',
'over 1 year',
'up to 30 days (including demand deposits)',
'31 to 90 days',
'91 to 180 days',
'181 days to 1 year',
'up to 1 year (including demand deposits)',
'1 to 3 years',
'over 3 years',
'over 1 year']

#### Loop over the dictionary of dataframes and extract the data needed into 2 lists for further calculations
#### Data mangling in the loop:
1. Remove new lines and white space from the month (first) column
2. Filter out lines which are not months defined in the first dataset index based on Deposit_amounts.xlsx file
3. Replace - with NaNs
4. Assign the MultiIndex columns to the dataset
5. Extract the first table from the dataset (weighted average annual interest rates on deposits in the reporting month and are calculated based on annual interest rates, which are stated in deposit agreement and volume of borrowed funds in the reporting month.)
6. Extract the second table from the dataset (Share of deposits on each maturity, borrowed in the reporting month, in total volume of deposits in the reporting month, borrowed from individuals and nonfinancial organizations.)
7. Multiply the numbers in the second table with their respective deposit amount (individual/entity) for each month

In [21]:
dfDepositsAnnIntRate = list()
dfDepositsStructureIntRate = list()
for k in dfdepositsRatesBuckets:
    dfdepositsRatesBuckets[k][0] = dfdepositsRatesBuckets[k][0].str.strip() # 1
    dfdepositsRatesBuckets[k] = dfdepositsRatesBuckets[k][dfdepositsRatesBuckets[k][0].isin(dfDepositAmounts.index)] # 2
    dfdepositsRatesBuckets[k] = dfdepositsRatesBuckets[k].replace('-', np.nan) # 3
    dfdepositsRatesBuckets[k].columns = idx # 4
    dfdepositsRatesBuckets[k].columns = dfdepositsRatesBuckets[k].columns.set_levels(cols, level=1) # 4
    tmpDf1 = dfdepositsRatesBuckets[k].ix[:6] # 5
    tmpDf1.set_index(tmpDf1.columns[0], inplace=True)
    tmpDf1.index.name = 'month'
    dfDepositsAnnIntRate.append(tmpDf1/100)
    tmpDf2 = dfdepositsRatesBuckets[k].ix[7:] # 6
    tmpDf2.set_index(tmpDf2.columns[0], inplace=True)
    tmpDf2 = tmpDf2/100
    if k == 0:
        curr = 'rub'
    else:
        curr = 'usd'
    # 7:
    tmpDf2_1 = tmpDf2.iloc[:, tmpDf2.columns.get_level_values(0)=='individual'].multiply(dfDepositAmounts['individual_{}'.format(curr)], axis=0)
    tmpDf2_2 = tmpDf2.iloc[:, tmpDf2.columns.get_level_values(0)=='entity'].multiply(dfDepositAmounts['entities_{}'.format(curr)], axis=0)
    dfDepositsStructureIntRate.append(tmpDf2_1.join(tmpDf2_2).round())

#### Calculate the number of days in a month based on the deposit amounts dataset date

In [22]:
daysInMonth = pd.to_numeric((dfDepositAmounts.dt + MonthEnd(1)).apply('{:%d}'.format))

In [23]:
daysInMonth = daysInMonth.to_frame()

#### Add order column to the dataset of days in months (Used later for sorting)

In [24]:
daysInMonth['order'] = [r + 1 for r in range(len(daysInMonth))]

In [25]:
daysInMonth

,dt,order
month,,
January,31,1
February,29,2
March,31,3
April,30,4
May,31,5
June,30,6
July,31,7
August,31,8


### 5. Extract the monthly avg weighted fx rates from the url and create a dataset out of them 

In [26]:
r  = requests.get(url)
data = r.text

In [27]:
soup = BeautifulSoup(data, 'lxml')

#### The relevant data we need is in a table with id table28

In [28]:
table28 = soup.find(id='table28')

In [29]:
type(table28)

bs4.element.Tag

#### Loop over the 1 and the 6 rows of the table and extract the months names and their respective fx rates

In [30]:
header = list()
info = list()
for n, tr in enumerate(table28.findAll('tr')):
    if n in (0,):
        # print(n, tr)
        for td in tr.findAll('td'):
            header.append(td.getText())
    elif n in (5,):
        for td in tr.findAll('td'):
            info.append(td.getText())

#### Remove . from months name and zip together the months and their fx rates and create a dataframe out of the zip then replace empty strings with NaN and drop the empty fields

In [31]:
zipped = list(zip([h.strip().replace('.', '') for h in header], [h.strip() for h in info]))

In [32]:
webRates = pd.DataFrame.from_records(zipped, columns=['month_short', 'rate']).replace('', np.nan).dropna()

#### Filter out months starting with Q (as they are not months but quaters) then convert the rates to a number. Conversion is needed because everything coming from a website is considered text

In [33]:
webRates = webRates[~webRates.month_short.str.startswith('Q')]
webRates['rate'] = pd.to_numeric(webRates.rate)

In [34]:
# for z in zip(webRates.index, daysInMonth.index):
#     print(z)

In [35]:
daysInMonth.index[daysInMonth.index.str.startswith('Jan')][0]

'January'

#### Map the full month name from the first dataset with the months short name from the url, set it as index and delete the moths short name

In [36]:
webRates['month'] = webRates.month_short.apply(lambda x: daysInMonth.index[daysInMonth.index.str.startswith(x)][0])

In [37]:
webRates.set_index('month', inplace=True)

In [38]:
del(webRates['month_short'])

In [39]:
webRates

,rate
month,
January,83.03
February,85.88
March,78.23
April,75.58
May,74.26
June,73.33
July,71.23
August,72.78


### 6. Create and excel with calculations applied

In [40]:
writer = pd.ExcelWriter('files/depositsResult.xlsx')

#### Loop over the deposits structures datasets in different currencies (rub, usd)
1. Take the annual weighted interest rate and divide it by 365 then multiply it by the days in a month 
2. Multiply the deposit structures with the result of the above operation
3. If currency is RUB divide by the usd exchange rate to get the USD equivalent
4. Add order column to the dataset and order by it then drop it and write dataset as a sheet to the excel file
5. Save the excel file

In [42]:
for n, df in enumerate(dfDepositsStructureIntRate):
    t = None
    if n == 0:
        t = 'RUB'
    else:
        t = 'USD'
    df2 = dfDepositsAnnIntRate[n].divide(365, axis=0).multiply(daysInMonth.dt, axis=0) # 1
    df = df.multiply(df2, axis=0).round() # 2
    if t == 'RUB':
        df = df.divide(webRates.rate, axis=0).round() # 3
    df['entity', 'order'] = daysInMonth['order'] # 4
    df = df.sort_values(('entity', 'order')) # 4
    df.drop('order', axis=1, level=1, inplace=True) # 4
    df.to_excel(writer,'sheet %s' % t)
writer.save() # 5

### Et voila!